Training a decoder only transformer for generating Elon Musk tweets

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Pre-processing

Loading Elon Tweets (test dataset), cleaning up, and tokenizing with dict representing the vocab, also np arr will be saved with all the tweet combination lengths 

In [ ]:
df = pd.read_csv('TweetsElonMusk.csv')
df = df['tweet']

# number of df rows
n = len(df)

# take all rows and concat them into one string
text = ''
for i in range(n):
    text += df[i]

# find unique chars in text
set_of_unique_chars = set(text)

# create a dictionary of unique chars, mapping each char to an int
char_to_int = {}

for i, char in enumerate(set_of_unique_chars):
    char_to_int[char] = i

# add 395 as <sos> token, 396 as <eos> token, 397 as <pad> token
char_to_int['<sos>'] = 395
char_to_int['<eos>'] = 396
char_to_int['<pad>'] = 397

# find row with longest string
max_len = 0
for i in range(n):
    if len(df[i]) > max_len:
        max_len = len(df[i])

elon_data = np.zeros((n, max_len + 2))

print(elon_data.shape)

# 395 is the sos token and 396 is the eos token, 397 is the padding token
for i in range(n):
    elon_data[i][0] = 395
    for j in range(len(df[i])):
        elon_data[i][j + 1] = char_to_int[df[i][j]]
    elon_data[i][len(df[i]) + 1] = 396
    elon_data[i][len(df[i]) + 2:] = 397

# save to np file
np.save('elon_data.npy', elon_data)

In [ ]:
print(char_to_int['<pad>'])

functions for converting tokens into chars and chars into tokens

In [ ]:
def tokens_to_chars(tokens):
    chars = []
    for token in tokens:
        for char in char_to_int:
            if char_to_int[char] == token:
                chars.append(char)
    return chars

def chars_to_tokens(chars):
    tokens = []
    for char in chars:
        tokens.append(char_to_int[char])
    return tokens

architecture parameters

In [ ]:
layers = 3
epochs = 10
batch_size = 100
embedding_dim = 64
dict_size = len(char_to_int)
attention_dim = 16
feed_forward_dim = embedding_dim
num_heads = 8 

## Decoder Architecture 

In [261]:
class Decoder(nn.Module):
    def __init__(self,layers, epochs, batch_size, embedding_dim, dict_size, feed_forward_dim, num_heads, attention_dim):
        super(Decoder, self).__init__()
        self.layers = layers
        self.epochs = epochs
        self.batch_size = batch_size
        self.embedding_dim = embedding_dim
        self.dict_size = dict_size
        self.feed_forward_dim = feed_forward_dim
        self.num_heads = num_heads
        self.attention_dim = attention_dim

        self.embedding = nn.Embedding(num_embeddings=dict_size+1, embedding_dim=embedding_dim)
        
        self.decoder_layer_1 = DecoderLayer(self.embedding_dim, self.attention_dim, self.feed_forward_dim, self.num_heads)
        self.decoder_layer_2 = DecoderLayer(self.embedding_dim, self.attention_dim, self.feed_forward_dim, self.num_heads)
        self.decoder_layer_3 = DecoderLayer(self.embedding_dim, self.attention_dim, self.feed_forward_dim, self.num_heads)

        self.linear = nn.Linear(in_features=embedding_dim, out_features=dict_size)
        self.softmax = nn.Softmax(dim=1)


    def forward(self, x):
        x = self.embedding(x)
        positional_encodings = self.positional_encoding(len(x), self.embedding_dim)

        x += positional_encodings
        
        x = self.decoder_layer_1.forward(x)
        x = self.decoder_layer_2.forward(x)
        x = self.decoder_layer_3.forward(x)

        x = self.linear(x)
        x = self.softmax(x)

        return x 

    ## This was generated by chat gpt-3, so hopefully it works
    def positional_encoding(self, seq_len, embedding_dim):
        positions = np.arange(seq_len)[:, np.newaxis]
        angles = np.power(10000, -(2 * (np.arange(embedding_dim) // 2) / embedding_dim))
        angles = angles[np.newaxis, :]

        positional_encodings = positions * angles

        # Apply sine to even indices in the array; 2i
        positional_encodings[:, 0::2] = np.sin(positional_encodings[:, 0::2])

        # Apply cosine to odd indices in the array; 2i+1
        positional_encodings[:, 1::2] = np.cos(positional_encodings[:, 1::2])

        positional_encodings = torch.from_numpy(positional_encodings).float()
        return positional_encodings

class DecoderLayer(nn.Module):    
    def __init__(self, embedding_dim, attention_dim, feed_forward_dim, num_heads):
        super(DecoderLayer, self).__init__()
        self.multi_head_attention = MultiHeadAttention(embedding_dim=embedding_dim, attention_dim=attention_dim, num_heads=num_heads)
        self.layer_norm_1 = nn.LayerNorm(embedding_dim)
        self.layer_norm_2 = nn.LayerNorm(embedding_dim)
        self.feed_forward_1 = nn.Linear(in_features=embedding_dim, out_features=feed_forward_dim)
        self.feed_forward_2 = nn.Linear(in_features=feed_forward_dim, out_features=embedding_dim)

    def forward(self, x):
        attention = self.multi_head_attention.forward(x)
        x = self.layer_norm_1(x + attention)
        feed_forward = self.feed_forward_2(F.relu(self.feed_forward_1(x)))
        x = self.layer_norm_2(x + feed_forward)
        return x 

class MultiHeadAttention(nn.Module):
    def __init__(self, embedding_dim, attention_dim, num_heads):
        super(MultiHeadAttention, self).__init__()

        self.num_heads = num_heads
        self.attention_dim = attention_dim
        self.embedding_dim = embedding_dim

        # linear transform from embedding -> attention_dim 
        # later divide into attention heads 
        self.Q = nn.Linear(in_features=embedding_dim, out_features=attention_dim * num_heads)
        self.K = nn.Linear(in_features=embedding_dim, out_features=attention_dim * num_heads)
        self.V = nn.Linear(in_features=embedding_dim, out_features=attention_dim * num_heads)

        # reduce the attention heads back to the embedding dim size 
        self.Reduction = nn.Linear(in_features=attention_dim * num_heads, out_features=embedding_dim)

    def forward(self, x):
        Q_vectors = self.Q(x)
        K_vectors = self.K(x)
        V_vectors = self.V(x)

        # this results in the shape sequence length, num_heads, attention_dim
        Q_vectors = torch.reshape(Q_vectors, (x.shape[0], self.num_heads, self.attention_dim))
        K_vectors = torch.reshape(K_vectors, (x.shape[0], self.num_heads, self.attention_dim))
        V_vectors = torch.reshape(V_vectors, (x.shape[0], self.num_heads, self.attention_dim))

        # now reshape into atten num heads, sequence length, attention dim
        Q_vectors = Q_vectors.transpose(0,1)
        K_vectors = K_vectors.transpose(0,1)
        V_vectors = V_vectors.transpose(0,1)

        # dot product of Q and K but not first dimension
        attention_scores = torch.matmul(Q_vectors, K_vectors.transpose(1,2))
        attention_scores = attention_scores / np.sqrt(self.attention_dim)

        # now the shape is attention head, sequence length, softmax values 

        # softmax over attention dim 
        attention_scores = nn.Softmax(dim=2)(attention_scores)

        # Multiply the value vectors by the attention scores {possible point of error}
        weighted_sum = torch.matmul(attention_scores, V_vectors)

        # Concatenate the heads back together
        concatenated = weighted_sum.transpose(0, 1).reshape(x.shape[0], -1, self.attention_dim * self.num_heads)

        # Linearly transform the concatenated vectors back to the embedding size
        output = self.Reduction(concatenated)

        # remove the middle 1 dimension
        output = torch.squeeze(output, dim=1)

        return output

# Training 

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# Define a custom dataset class
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Convert the data to integer tensors
        tensor_data = torch.tensor(self.data[index], dtype=torch.int32)

        return tensor_data

# load elon data npy
elon_data = np.load('elon_data.npy')

# Convert your data to integer tensors
int_data = [[int(char) for char in example] for example in elon_data]

# Create an instance of the custom dataset
dataset = MyDataset(int_data)

# Create the dataloader using the dataset
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [257]:
def softmax_to_char(softmax):
    return np.argmax(softmax)

# output dist vs actual disturbution
def loss_function(predictions, targets):
    return nn.CrossEntropyLoss()(predictions, targets)

def target_seq_to_distribution(value):
    distribution = torch.zeros((dict_size))
    value = int(value) -1
    distribution[value] = 1.0
    return distribution

In [263]:
import torch.utils.data as data
from torch.utils.data import DataLoader
import torch.optim as optim

decoder = Decoder(layers=layers, epochs=epochs, batch_size=batch_size, embedding_dim=embedding_dim, dict_size=dict_size, feed_forward_dim=feed_forward_dim, num_heads=num_heads, attention_dim=attention_dim)

# Create an optimizer for the Decoder model
optimizer = optim.Adam(decoder.parameters(), lr=0.001)

# Associate the optimizer with the Decoder object
decoder.optimizer = optimizer

# random number generator 
torch.manual_seed(0)

for epoch in range(epochs):
    for i_batch, batch in enumerate(dataloader):
        # torch random between 0 and 426
        stop_seq = torch.randint(0,(batch.shape[1]-1), (1,))[0]
        next_token = stop_seq + 1

        # get the input and target sequences
        input_seq = batch[:, :stop_seq]
        target_seq = batch[:, stop_seq:next_token]

        # zero out the gradients
        decoder.optimizer.zero_grad()

        output = []

        # shitty implementation, but it works for now (no batching implemented in actual transformer)
        for i in range(batch_size):
            x = decoder.forward(input_seq[i])
            # get last output 
            output.append(x[-1])

        # convert to tensor
        output_tensor = torch.stack(output)
        
        target = []
        for i in range(batch_size):
            target.append(target_seq_to_distribution(target_seq[i]))
        
        target_seq = torch.stack(target)

        print(output_tensor.shape)
        print(target_seq.shape)

        # Compute the loss
        loss = loss_function(output_tensor, target_seq)

        # Backpropagation
        loss.backward()

        # Update the weights
        optimizer.step()

torch.Size([100, 397])
torch.Size([100, 397])
tensor(5.9853, grad_fn=<DivBackward1>)
